In [1]:
import os
os.chdir('/Users/alexascunceparis/Desktop/BSC/immuno_project/TCRranker')

In [3]:
#Import libraries
import os
import pandas as pd
import numpy as np
from Bio import PDB


residue_mapping = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D',
    'CYS': 'C', 'GLU': 'E', 'GLN': 'Q', 'GLY': 'G',
    'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K',
    'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S',
    'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'
}

def extract_contacts(pdb_files, chain_dict, distance=5):
    """
    Extract contacting atoms between specific chains based on a distance threshold.
    
    Args:
        pdb_files (list of str): List of file paths to PDB files.
        chain_dict (dict): Dictionary containing chains for each PDB ID.
        distance (float): Distance threshold for considering contacts.
    
    Returns:
        pd.DataFrame: DataFrame containing contacts with columns ['pdb_id', 'chain_from', 'chain_to', 
                                                                 'residue_from', 'residue_to', 
                                                                 'resid_from', 'resid_to', 
                                                                 'atom_from', 'atom_to', 'dist'].
    """
    contacts = []
    
    for pdb_file in pdb_files:
        pdb_id = os.path.basename(pdb_file).split('.')[0]
        print(f"Extracting contacts from {pdb_id}")
        parser = PDB.PDBParser(QUIET=True)
        structure = parser.get_structure(pdb_id, pdb_file)
        model = structure[0]

        # Retrieve chains of interest from chain_dict
        chains = chain_dict.get(pdb_id)
        if not chains:
            continue
        
        # Define pairs of chains to extract contacts
        chain_pairs = [
            (chains['tcra_chain'], chains['mhc_chain']),
            (chains['tcrb_chain'], chains['mhc_chain']),
            (chains['tcra_chain'], chains['peptide_chain']),
            (chains['tcrb_chain'], chains['peptide_chain'])
        ]
        
        for chain_from_id, chain_to_id in chain_pairs:
            if chain_from_id and chain_to_id:  # Ensure both chains are defined
                try:
                    chain_from = model[chain_from_id]
                    chain_to = model[chain_to_id]
                except KeyError:
                    print(f"Chain not found in {pdb_id}: {chain_from_id} or {chain_to_id}")
                    continue
                
                residues_from = list(chain_from.get_residues())
                residues_to = list(chain_to.get_residues())
                
                for residue_from in residues_from:
                    for residue_to in residues_to:
                        atoms_from = list(residue_from.get_atoms())
                        atoms_to = list(residue_to.get_atoms())
                        
                        for atom_from in atoms_from:
                            for atom_to in atoms_to:
                                dist = np.linalg.norm(atom_from.coord - atom_to.coord)
                                if dist <= distance:  # Threshold for contact
                                    res_from_single = residue_mapping.get(residue_from.get_resname(), 'X')  # 'X' for unknown residues
                                    res_to_single = residue_mapping.get(residue_to.get_resname(), 'X')  # 'X' for unknown residues
                                    
                                    # Original contact
                                    contacts.append([
                                        pdb_id, 
                                        chain_from.get_id(), 
                                        chain_to.get_id(), 
                                        res_from_single, 
                                        res_to_single, 
                                        residue_from.get_id()[1], 
                                        residue_to.get_id()[1], 
                                        atom_from.get_id(), 
                                        atom_to.get_id(), 
                                        dist
                                    ])
    
    return pd.DataFrame(contacts, columns=['pdb_id', 'chain_from', 'chain_to', 'residue_from', 'residue_to', 'resid_from', 'resid_to', 'atom_from', 'atom_to', 'dist'])

In [2]:
pdb_dir = './pdb_files/'  
pdb_files = [os.path.join(pdb_dir, f) for f in os.listdir(pdb_dir) if f.endswith('.pdb')]

chain_dict = {}
general_df = pd.read_csv('./structures_annotation/general.txt', sep='\t')

for pdb_id, group in general_df.groupby('pdb.id'):
    chains = {
        'tcra_chain': None,
        'tcrb_chain': None,
        'peptide_chain': None,
        'mhc_chain': None
    }
    
    for _, row in group.iterrows():
        if row['chain.component'] == 'TCR' and row['chain.type'] == 'TRA':
            chains['tcra_chain'] = row['chain.id']
        elif row['chain.component'] == 'TCR' and row['chain.type'] == 'TRB':
            chains['tcrb_chain'] = row['chain.id']
        elif row['chain.component'] == 'PEPTIDE':
            chains['peptide_chain'] = row['chain.id']
        elif row['chain.component'] == 'MHC' and row['chain.supertype'] == 'MHCI' and row['chain.type'] == 'MHCa':
            chains['mhc_chain'] = row['chain.id']
        
    chain_dict[pdb_id] = chains 


for pdb_file in pdb_files:
    pdb_id = os.path.basename(pdb_file).split('.')[0]
    output_file = f'./contact_maps/{pdb_id}_contacts.csv'
    if os.path.exists(output_file):
        print(f"File {output_file} exists, omiting...")
        continue
    else:
        contacts_df = extract_contacts([pdb_file], chain_dict)
    contacts_df.to_csv(output_file, index=False)
    print(f"Saved contacts in {output_file}.")

NameError: name 'pd' is not defined

#### SEE IF ALL CONTACT MAPS CAN BE USED

In [5]:
import os
import pandas as pd

def validate_chain_columns(csv_file):
    """
    Verifica que las columnas 'chain_from' y 'chain_to' del archivo CSV contengan exactamente 2 cadenas únicas cada una.

    Args:
    - csv_file (str): Ruta al archivo CSV.

    Returns:
    - bool: True si cumple con la condición, False si no.
    - dict: Diccionario con las cadenas encontradas en 'chain_from' y 'chain_to'.
    """
    # Cargar el archivo CSV
    df = pd.read_csv(csv_file)

    # Verificar si las columnas 'chain_from' y 'chain_to' existen
    if 'chain_from' not in df.columns or 'chain_to' not in df.columns:
        raise ValueError(f"Archivo {csv_file} no contiene las columnas 'chain_from' o 'chain_to'.")

    # Obtener los valores únicos en las columnas 'chain_from' y 'chain_to'
    unique_chain_from = df['chain_from'].unique()
    unique_chain_to = df['chain_to'].unique()

    # Validar que ambas columnas tengan exactamente 2 cadenas únicas
    is_valid = len(unique_chain_from) == 2 and len(unique_chain_to) == 2

    # Devolver el resultado y las cadenas encontradas
    return is_valid, {'chain_from': unique_chain_from, 'chain_to': unique_chain_to}

def check_contact_maps(directory):
    """
    Recorre la carpeta contact_maps y verifica cada archivo CSV para asegurar que 'chain_from' y 'chain_to' tengan 2 cadenas únicas.

    Args:
    - directory (str): Directorio que contiene los archivos CSV.
    """
    # Verificar que el directorio existe
    if not os.path.isdir(directory):
        raise ValueError(f"El directorio {directory} no existe.")

    # Listar todos los archivos en el directorio
    for file_name in os.listdir(directory):
        if file_name.endswith('_contacts.csv'):
            file_path = os.path.join(directory, file_name)
            try:
                is_valid, chains = validate_chain_columns(file_path)
                if is_valid:
                    print(f"Archivo {file_name} es válido.")
                else:
                    print(f"Archivo {file_name} no es válido. Cadenas encontradas: {chains}")
            except Exception as e:
                print(f"Error procesando {file_name}: {e}")

check_contact_maps('./contact_maps_testing/')

Archivo filtered_3vxu_contacts.csv es válido.
Archivo filtered_6p64_contacts.csv es válido.
Archivo filtered_7byd_contacts.csv es válido.
Archivo filtered_6amu_contacts.csv es válido.
Archivo filtered_8dnt_contacts.csv es válido.
Archivo filtered_3kps_contacts.csv es válido.
Archivo filtered_4g9f_contacts.csv es válido.
Archivo filtered_8qfy_contacts.csv es válido.
Archivo filtered_8d5q_contacts.csv es válido.
Archivo filtered_7n6e_contacts.csv es válido.
Archivo filtered_5sws_contacts.csv es válido.
Archivo filtered_3sjv_contacts.csv es válido.
Archivo filtered_5nmg_contacts.csv es válido.
Archivo filtered_7n1e_contacts.csv es válido.
Archivo filtered_8enh_contacts.csv es válido.
Archivo filtered_5jzi_contacts.csv es válido.
Archivo filtered_5men_contacts.csv es válido.
Archivo filtered_5eu6_contacts.csv es válido.
Archivo filtered_6bj3_contacts.csv es válido.
Archivo filtered_7jwj_contacts.csv es válido.


#### FILTER X RESIDUES

In [ ]:
import pandas as pd
import os

# Carpeta donde están los archivos CSV
folder_path = 'contact_maps'

# Iterar sobre todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    # Verificar si el archivo es un CSV
    if filename.endswith('.csv'):
        # Ruta completa al archivo
        file_path = os.path.join(folder_path, filename)
        
        # Cargar el archivo CSV
        df = pd.read_csv(file_path)

        # Filtrar las filas donde 'residue_from' o 'residue_to' sean 'X'
        filtered_df = df[(df['residue_from'] != 'X') & (df['residue_to'] != 'X')]

        # Guardar el nuevo DataFrame en un nuevo archivo CSV
        filtered_file_path = os.path.join(folder_path, f'filtered_{filename}')
        filtered_df.to_csv(filtered_file_path, index=False)

        print(f'Filtrado {filename} y guardado como {filtered_file_path}')